#### Leveraging glycophenotypes for diagnosis of human diseases: example of Fucosidosis


We hypothesize that expanding the representation of glycophenotypes in the HPO and their use in disease annotations could improve phenotype-based comparisons in the Monarch Initiative. Here we perform an analysis with phenotypes associated with the disease fucosidosis (MONDO:0009254). First, we generate 100 "simulated" disease phenotype profiles by randomly sampling 10 of the 74 phenotypes associated with fucosidosis.  In a second set, for each profile, we randomly replaced two phenotypes with two glycophenotypes known to be associated with fucosidosis. Both sets are compared using the OwlSim algorithm from the Monarch Initiative, which given a list of phenotypes, ranks candidate diseases based on phenotypic similarity.

This notebook walks through this analysis, first where the 10 sampled non glycophenotypes and 2 glycophenotypes are provided, and a second run where the entire analysis is randomized.

##### Ontologies, data, and tools

###### Ontologies
This analysis uses the following versions of ontologies:

Human Phenotype Ontology (HPO): http://purl.obolibrary.org/obo/hp/releases/2018-12-21/hp.owl

Monarch Disease Ontology (MONDO): http://purl.obolibrary.org/obo/mondo/releases/2018-12-02/mondo.owl

###### Disease to phenotype annotations

The [HPO disease to phenotype annotation file](http://compbio.charite.de/jenkins/job/hpo.annotations/lastStableBuild/artifact/misc/phenotype_annotation.tab) is not versioned; therefore, a snapshot of is provided alongside this notebook,

_HPO annotations_: ./input/phenotype_annotation.tab

Disease identifiers were merged using MONDO, and annotations updated with merged IDs,

_Disease to phenotype annotations_: ./input/mondo_hp.tsv

###### OwlSim
The official Monarch Initiative OwlSim endpoint is available via:
https://monarchinitiative.org/owlsim/

Alternatively, we provide a docker file to run owlsim locally with the above ontology and annotation files. To run:

    cd owlsim-docker
    docker build ./ -t owlsim-slim
    docker run -d -p 9031:9031 --name owlsim owlsim-slim
    
    # Test that owlsim is running (may take 60 seconds to start)
    # http://localhost:9031/compareAttributeSets?a=HP:0010539&b=HP:0006989&b=HP:0000219&b=HP:0000248
    
    # To stop the container
    docker stop owlsim


##### Running this notebook
This is a python3 notebook and assumes the environment running jupyter contains the following python libs:
- requests
- rdflib

